<a href="https://colab.research.google.com/github/reyllama/TTIO/blob/master/05_IndustryAttached.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.autograd import Variable
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [22]:
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
code_df.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
1,GS,78930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
2,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
3,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등",2018-06-12,12월,"권순호, 정경구",http://www.hdc-dvp.com,서울특별시
4,LG이노텍,11070,전자부품 제조업,기타 전자부품 제조업,2008-07-24,12월,정철동,http://www.lginnotek.co.kr,서울특별시


In [23]:
core_df = code_df[['회사명', '종목코드', '업종']].rename(columns={'회사명': 'name', '종목코드':'stock_id', '업종': 'industry'})
core_df.head()

,name,stock_id,industry
0,DSR,155660,1차 비철금속 제조업
1,GS,78930,기타 금융업
2,GS글로벌,1250,상품 종합 도매업
3,HDC현대산업개발,294870,건물 건설업
4,LG이노텍,11070,전자부품 제조업


In [24]:
data = pd.read_csv("/content/drive/My Drive/stocks.csv")
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,stock_id,name,date,opening_price,highest_price,lowest_price,closing_price,trading_volume
0,000020,동화약품,2016-01-04,8130,8150,7920,8140,281440
1,000020,동화약품,2016-01-05,8040,8250,8000,8190,243179
2,000020,동화약품,2016-01-06,8200,8590,8110,8550,609906
3,000020,동화약품,2016-01-07,8470,8690,8190,8380,704752
4,000020,동화약품,2016-01-08,8210,8900,8130,8770,802330


In [25]:
core_df['stock_id'] = core_df['stock_id'].astype(str)
data['stock_id'] = data['stock_id'].astype(str)

core_df.head()

,name,stock_id,industry
0,DSR,155660,1차 비철금속 제조업
1,GS,78930,기타 금융업
2,GS글로벌,1250,상품 종합 도매업
3,HDC현대산업개발,294870,건물 건설업
4,LG이노텍,11070,전자부품 제조업


In [32]:
temp = data[data['name'].isna()].groupby('stock_id')['date'].count()==979
temp[temp].index

Index(['103130', '11160', '122800', '123260', '159650', '1799', '197210',
       '27390', '30270', '54340', '77280', '83370', '83380', '900040',
       '94190'],
      dtype='object', name='stock_id')

In [39]:
total = data[data['stock_id']=='005930'].shape[0]
t = data.groupby('stock_id')['date'].count()==total
complete = t[t].index
data = data[data['stock_id'].isin(complete)]
data = data.dropna(subset=['name'])

merged = data.merge(core_df, on='name', how='left')
merged.isna().sum().sort_values()

stock_id_x             0
name                   0
date                   0
opening_price          0
highest_price          0
lowest_price           0
closing_price          0
trading_volume         0
stock_id_y        162514
industry          162514
dtype: int64

In [41]:
merged[merged['industry'].isna()]

,stock_id_x,name,date,opening_price,highest_price,lowest_price,closing_price,trading_volume,stock_id_y,industry
4895,000075,삼양홀딩스우,2016-01-04,68700,69200,66200,68300,911,NaN,NaN
4896,000075,삼양홀딩스우,2016-01-05,68200,69400,67700,69400,452,NaN,NaN
4897,000075,삼양홀딩스우,2016-01-06,69100,72600,68800,72400,1792,NaN,NaN
4898,000075,삼양홀딩스우,2016-01-07,70600,72800,70600,72400,810,NaN,NaN
4899,000075,삼양홀딩스우,2016-01-08,72900,74800,71600,74800,939,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1849326,900140,엘브이엠씨홀딩스,2019-12-23,4300,4300,4175,4185,114713,NaN,NaN
1849327,900140,엘브이엠씨홀딩스,2019-12-24,4185,4185,3900,4030,151428,NaN,NaN
1849328,900140,엘브이엠씨홀딩스,2019-12-26,4035,4300,4030,4205,128054,NaN,NaN
1849329,900140,엘브이엠씨홀딩스,2019-12-27,4220,4240,4135,4180,62532,NaN,NaN


In [43]:
merged = merged.drop(columns=['stock_id_y']).rename(columns={'stock_id_x': 'stock_id'})
merged.to_csv("stocks_merged.csv")